# Integration Demo

Following is a step-by-step guide comparing the two integration approaches use in the GaussianNEGF package. All steps have been tested on a 20 core system with each step running in seconds except for the final IV step, which can take minutes.

## System Setup

We will be working with an ethane molecule:
<img src="ethane.png" alt="drawing" width="200"/>

**Basis set:** 6-31g(d,p) -  C atoms will have 15 basis functions and H will have 5

**Functional:** B3LYP Hydbrid functional

For this system we will use a diagonal self energy matrix, with $\Gamma_i=0.1$ ($\Sigma_i = -0.05j$). Because this is an energy independent self-energy, we can first test the system using the `NEGF()` type object from the `scf.py` file.

## Energy Independent Approach
To set up the system lets first import the packages and initialize the ethane system:

In [ ]:
from gauNEGF.scf import NEGF
from gauNEGF.density import *
from gauNEGF.surfGBethe import *
from matplotlib import pyplot as plt

negf = NEGF(fn='ethane', func='b3lyp', basis='6-31g(d,p)', spin='r')
print(negf.bar.ian)

The gaussian interface `QCBinAr` object is stored in `negf.bar`.

The atomic numbers for each atom are stored in `negf.bar.ian`, which are C, C, H, H, H, H, H, H

Now we can attach the contacts to both carbon atoms:

In [ ]:
negf.setSigma([1], [2], -0.05j)

And set the voltage to zero, setting the Fermi energy to the default value $\left(\frac{E_H + E_L}2\right)$:

In [ ]:
negf.setVoltage(0)

Now let's run a quick NEGF-DFT job setting the convergence to $10^{-4}$ and the mixing factor to 0.05:

In [ ]:
nIter, neList, Elist = negf.SCF(1e-3, 0.1)
# Plot convergence of DFT Energy
plt.plot(nIter, Elist)
plt.xlabel('Iteration Number')
plt.ylabel('Total DFT Energy (eV)')

A quick check of the SCF run can be done by looking at the energy level occupation list printed at the end of the run. As can be seen, the Fermi energy ends up close to the HOMO energy, and the occupation of HOMO energy and below is close to 1.0 whereas all higher energy levels have occupations near 0.0. We can also look at the main diagonal of the density matrix to understand how the atomic orbitals are occupied, multiplying by 2 to account for spin:

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(10,5))
rho = negf.P * 2

# Count number of basis functions per atom:
atlocs = np.cumsum([(negf.bar.ibfatm==i+1).sum() for i in range(negf.bar.natoms-1)])
# Plot dividing lines between each atom
for a in atlocs:
    ax1.axvline(a, color='k', linestyle=':')
    ax2.axvline(a, color='k', linestyle=':')

# Plot real part of the diagonal of the density matrix
ax1.bar(np.arange(len(negf.P))+0.5, np.diag(rho.real))
ax1.set_title(r'$\mathbb{Re}\left[\rho_{ii}\right]$')
# Plot imaginary part of the diagonal of the density matrix
ax2.bar(np.arange(len(negf.P))+0.5, np.diag(rho.imag))
ax2.set_title(r'$\mathbb{Im}\left[\rho_{ii}\right]$')

To integrate the energy independent case, an analytical solution was used by solving the integral equation:

$$
    2 \pi \hat{\rho}_j = \hat{S}^{-\frac12}\left[ \int_{-\infty}^{\infty} f_j(E) \left(\sum_n \frac{|n\rangle}{E - \epsilon_n} \right) \langle n|\bar{\Gamma}_j |n'\rangle \right. \nonumber \\
   \left.  \left(\sum_{n'} \frac{\langle n'|}{E - \epsilon_n^\dagger} \right) dE  \right] \hat{S}^{-\frac12} \ \ \ 
   $$

where

$$
\bar{\Gamma}_j = \hat{S}^{-\frac12}\hat{\Gamma}_j\hat{S}^{-\frac12}\\
\bar{G} = (E \hat{I} - \bar{F})^{-1}\\
\bar{F} =  \hat{S}^{-\frac12}(\hat{F} + \hat{\Sigma}_L + \hat{\Sigma}_R)  \hat{S}^{-\frac12}\\
\bar{F}|n\rangle= \epsilon_n |n\rangle
$$

and $j$ represents the contribution from the jth contact. Note that we will assume that the temperature is zero, so the Fermi function ($f_j(E)$) can be replaced with a finite integral up to $E_{F, j}$. 

In our case, two contacts are applied (the left and right) so

$$
\hat{\rho} = \hat{\rho}_L + \hat{\rho}_R
$$

Since $\bar{\Gamma}_j$ is constant with respect to energy, the solution can be solved without numerical integration, calculating the value of the analytical solution at the integration limits (some $E_{min}$ to represent negative infinity and the fermi energy $E_F$). This matrix math only requires a single diagonalization of $\bar{F}$ to get the values of $\epsilon_n$ and a single matrix exponent to get $\hat{S}^{-\frac12}$. To test this math, we can use numerical integration methods that allow for an energy dependent $\hat{\Gamma}_j$.

## ENERGY DEPENDENT APPROACH

First, let us import the energy dependent packages and set up the same system:

In [ ]:
from gauNEGF.scfE import NEGFE

negf2 = NEGFE(fn='ethane', func='b3lyp', basis='6-31g(d,p)', spin='r')
print(negf2.bar.ian)

Now, we can set the contact on our `NEGFE()` object for ethane:

In [ ]:
indsList = negf2.setSigma([1], [2], -0.05j)

And then we can set the voltage and calculate the energy mesh used for numerical integration (default parameters: tolerance 1e-4, mixing factor of 0.02 and 10 initial iterations):

In [ ]:
negf2.setVoltage(0.0)
#negf2.integralCheck()

Finally, we can run the SCF with the same parameters as before ($10^{-3}$, mixing of 0.1):

In [ ]:
nIter, neList, Elist = negf2.SCF(1e-3, 0.01)
# Plot convergence of DFT Energy
plt.plot(nIter, Elist)
plt.xlabel('Iteration Number')
plt.ylabel('Total DFT Energy (eV)')

This method use complex contour integration of the retarded Green's function to calculate the density matrix, based on the following equation.

$$
\hat{\rho} =  -\frac{1}{\pi}\mathbb{Im}\left[\int_{-\infty}^{E_F} \hat{G}^R(E) dE \right]      
$$

Note that this assumes that temperature is zero, as used in the energy-independent case.

## Comparison of Approaches

We can now compare the generated density matrices side by side:

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(10,5))
im = ax1.imshow(negf.P.real)
ax1.set_title(r'Analytical Integration $\rho$')
im2 = ax2.imshow(negf2.P.real)
ax2.set_title(r'Numerical Integration $\rho$')
fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
fig.colorbar(im2, cax=cbar_ax)

As a second check, we can calculate the density of states using each method. First, we need to import the `transport` package and generate the energy grid, then calculate DOS for each object:

In [ ]:
from gauNEGF.transport import *

Elist = np.linspace(-20, 20, 1000)
DOS1, _ = DOS(Elist+negf.fermi, negf.F, negf.S, negf.sigma1, negf.sigma2)
DOS2, _ = DOSE(Elist+negf2.fermi, negf2.F, negf2.S, negf2.g)

In [ ]:
plt.semilogy(Elist, DOS1, Elist, DOS2)
plt.legend(('Analytical Integration', 'Numerical Integration'))
plt.xlabel(r'$E-E_F$ (eV)')
plt.ylabel('Density of States')

## IV Characteristic Comparison

We can now use NEGF to sweep the voltage and calculate current across the molecule. This requires using the NEGF-DFT solver at each voltage point to calculate the transmission and then integrating to get the current. For $T=0 K$ the fermi function becomes a step function and the Landauer formula simplifies to:

$$
I(V)= \frac{q^2}{h}\int_{\mu - \frac{qV}{2}}^{\mu + \frac{qV}{2}} T(E, V) dE
$$

which is calculated after convergence using the `quickCurrent()` and `quickCurrentE()` functions from `transport.py` (_NOTE: This calculation can take some time to run_):

In [ ]:
# Voltage from 0 to 0.5 to -0.5 to 0
Vlist = list(np.arange(0.1, 0.5, 0.1))
Vlist += list(np.arange(0.5, -0.5, -0.1))
Vlist +=  list(np.arange(-0.5, 0.1, 0.1))
Ilist = []
IlistE =[]
for V in Vlist:
    print(f'SETTING VOLTAGE: {V} V')
    negf.setVoltage(V, fermi=negf.fermi)
    negf.SCF(1e-3, 0.02)
    negf2.setVoltage(V, fermi=negf.fermi)
    negf2.SCF(1e-3, 0.02)
    I = current(negf.F, negf.S, negf.sigma1, negf.sigma2, negf.fermi, V)
    I2 = currentE(negf2.F, negf2.S, negf2.g, negf2.fermi, V)
    Ilist.append(I)
    IlistE.append(I2)
    print(f'CALCULATED CURRENT Energy independent -  {I} A, Energy dependent - {I2} A')
print('IV COMPLETE!')


Now, as a final confirmation, we can plot these and make sure they match up:

In [ ]:
plt.plot(Vlist, [I*1e6 for I in Ilist], '-o')
plt.plot(Vlist, [I*1e6 for I in IlistE], '-o')
plt.xlabel('Voltage (V)')
plt.ylabel(r'Current ($\mu$A)')
plt.title('IV Characteristic')
plt.legend(('Energy Independent Calculation', 'Energy Dependent Calculation'))